In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 1000

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
torch.manual_seed(0)

In [13]:
        class Net(nn.Module):
                def __init__(self):
                        super().__init__()
                        self.conv1 = nn.Conv2d(3, 6, 5)
                        self.conv1_bn = nn.BatchNorm2d(6)
                        self.pool = nn.MaxPool2d(2, 2)
                        self.conv2 = nn.Conv2d(6, 10, 5)
                        self.conv2_bn = nn.BatchNorm2d(10)
                        self.fc1 = nn.Linear(10 * 5 * 5, 80)
                        self.fc1_bn = nn.BatchNorm1d(80)
                        self.fc2 = nn.Linear(80, 40)
                        self.fc2_bn = nn.BatchNorm1d(40)
                        self.fc3 = nn.Linear(40, 10)

                def forward(self, x):
                        x = self.pool(F.elu( self.conv1_bn(self.conv1(x))  ))
                        x = self.pool(F.elu( self.conv2_bn(self.conv2(x))  ))
                        x = torch.flatten(x, 1) # flatten all dimensions except batch
                        x = F.elu( self.fc1_bn(self.fc1(x)) )
                        x = F.elu( self.fc2_bn(self.fc2(x)) )
                        x = self.fc3(x)
                        return x

In [14]:
model = Net()

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [18]:
for i, data in enumerate(trainloader, 0):
    if i >= 1:
        break

    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss = loss.item()

    acc = (outputs.max(1)[1] == labels).sum().item()/labels.size(0)

    if epoch % 1 == 0:
        print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, 0 + 1, running_loss, acc ))
        
    print("\n\n\n\n\nPrint Gradient\n\n\n\n\n")
    lst = list(model.parameters())

    for i in range(len(lst)):
#         print(lst[i].grad.shape)
        print( "norm of grandient = ", torch.norm(lst[i].grad) )

print('Finished Training')

[1,     1] loss: 1.768 acc: 0.429





Print Gradient





norm of grandient =  tensor(0.1092)
norm of grandient =  tensor(5.1809e-07)
norm of grandient =  tensor(0.0122)
norm of grandient =  tensor(0.0089)
norm of grandient =  tensor(0.2165)
norm of grandient =  tensor(9.8646e-08)
norm of grandient =  tensor(0.0175)
norm of grandient =  tensor(0.0046)
norm of grandient =  tensor(0.3464)
norm of grandient =  tensor(7.1989e-09)
norm of grandient =  tensor(0.0136)
norm of grandient =  tensor(0.0057)
norm of grandient =  tensor(0.2253)
norm of grandient =  tensor(8.3488e-09)
norm of grandient =  tensor(0.0457)
norm of grandient =  tensor(0.0202)
norm of grandient =  tensor(0.2895)
norm of grandient =  tensor(0.0266)
[2,     1] loss: 1.765 acc: 0.431





Print Gradient





norm of grandient =  tensor(0.1071)
norm of grandient =  tensor(4.0165e-07)
norm of grandient =  tensor(0.0118)
norm of grandient =  tensor(0.0090)
norm of grandient =  tensor(0.2151)
norm of grandient =  tensor(7.0286

[13,     1] loss: 1.731 acc: 0.450





Print Gradient





norm of grandient =  tensor(0.1088)
norm of grandient =  tensor(4.2362e-07)
norm of grandient =  tensor(0.0121)
norm of grandient =  tensor(0.0090)
norm of grandient =  tensor(0.2053)
norm of grandient =  tensor(8.4695e-08)
norm of grandient =  tensor(0.0162)
norm of grandient =  tensor(0.0046)
norm of grandient =  tensor(0.3317)
norm of grandient =  tensor(7.2933e-09)
norm of grandient =  tensor(0.0135)
norm of grandient =  tensor(0.0057)
norm of grandient =  tensor(0.2140)
norm of grandient =  tensor(7.5687e-09)
norm of grandient =  tensor(0.0480)
norm of grandient =  tensor(0.0205)
norm of grandient =  tensor(0.2857)
norm of grandient =  tensor(0.0252)
[14,     1] loss: 1.728 acc: 0.452





Print Gradient





norm of grandient =  tensor(0.1074)
norm of grandient =  tensor(3.7669e-07)
norm of grandient =  tensor(0.0123)
norm of grandient =  tensor(0.0090)
norm of grandient =  tensor(0.2050)
norm of grandient =  tensor(8.95

[25,     1] loss: 1.696 acc: 0.460





Print Gradient





norm of grandient =  tensor(0.1008)
norm of grandient =  tensor(7.2942e-07)
norm of grandient =  tensor(0.0130)
norm of grandient =  tensor(0.0087)
norm of grandient =  tensor(0.1979)
norm of grandient =  tensor(9.2747e-08)
norm of grandient =  tensor(0.0149)
norm of grandient =  tensor(0.0046)
norm of grandient =  tensor(0.3207)
norm of grandient =  tensor(8.9587e-09)
norm of grandient =  tensor(0.0135)
norm of grandient =  tensor(0.0057)
norm of grandient =  tensor(0.2050)
norm of grandient =  tensor(9.4695e-09)
norm of grandient =  tensor(0.0499)
norm of grandient =  tensor(0.0209)
norm of grandient =  tensor(0.2818)
norm of grandient =  tensor(0.0241)
[26,     1] loss: 1.693 acc: 0.462





Print Gradient





norm of grandient =  tensor(0.1012)
norm of grandient =  tensor(4.9339e-07)
norm of grandient =  tensor(0.0130)
norm of grandient =  tensor(0.0087)
norm of grandient =  tensor(0.1978)
norm of grandient =  tensor(1.24

[37,     1] loss: 1.663 acc: 0.478





Print Gradient





norm of grandient =  tensor(0.1099)
norm of grandient =  tensor(3.6946e-07)
norm of grandient =  tensor(0.0128)
norm of grandient =  tensor(0.0090)
norm of grandient =  tensor(0.1925)
norm of grandient =  tensor(6.5785e-08)
norm of grandient =  tensor(0.0136)
norm of grandient =  tensor(0.0046)
norm of grandient =  tensor(0.3121)
norm of grandient =  tensor(8.0996e-09)
norm of grandient =  tensor(0.0136)
norm of grandient =  tensor(0.0056)
norm of grandient =  tensor(0.1978)
norm of grandient =  tensor(8.7934e-09)
norm of grandient =  tensor(0.0517)
norm of grandient =  tensor(0.0213)
norm of grandient =  tensor(0.2779)
norm of grandient =  tensor(0.0233)
[38,     1] loss: 1.660 acc: 0.482





Print Gradient





norm of grandient =  tensor(0.1093)
norm of grandient =  tensor(3.5704e-07)
norm of grandient =  tensor(0.0128)
norm of grandient =  tensor(0.0090)
norm of grandient =  tensor(0.1919)
norm of grandient =  tensor(8.63

[49,     1] loss: 1.631 acc: 0.486





Print Gradient





norm of grandient =  tensor(0.1049)
norm of grandient =  tensor(2.3391e-07)
norm of grandient =  tensor(0.0122)
norm of grandient =  tensor(0.0092)
norm of grandient =  tensor(0.1888)
norm of grandient =  tensor(1.0572e-07)
norm of grandient =  tensor(0.0126)
norm of grandient =  tensor(0.0045)
norm of grandient =  tensor(0.3059)
norm of grandient =  tensor(8.0800e-09)
norm of grandient =  tensor(0.0137)
norm of grandient =  tensor(0.0056)
norm of grandient =  tensor(0.1922)
norm of grandient =  tensor(9.8537e-09)
norm of grandient =  tensor(0.0534)
norm of grandient =  tensor(0.0218)
norm of grandient =  tensor(0.2745)
norm of grandient =  tensor(0.0226)
[50,     1] loss: 1.628 acc: 0.485





Print Gradient





norm of grandient =  tensor(0.1043)
norm of grandient =  tensor(7.2713e-07)
norm of grandient =  tensor(0.0124)
norm of grandient =  tensor(0.0090)
norm of grandient =  tensor(0.1883)
norm of grandient =  tensor(1.50

In [107]:
# nvar = 10160
# x = .1 * np.ones((nvar,1))
# x_torch = torch.from_numpy(x).cuda()
# torch.nn.utils.vector_to_parameters(x_torch, model.parameters())

lst = list(model.parameters())

for i in range(len(lst)):
    print(lst[i].grad.shape)
    print(lst[i].grad[0])

torch.Size([6, 3, 5, 5])
tensor([[[-0.0006, -0.0007, -0.0005, -0.0005, -0.0007],
         [-0.0005, -0.0006, -0.0006, -0.0006, -0.0008],
         [-0.0006, -0.0007, -0.0009, -0.0009, -0.0009],
         [-0.0008, -0.0010, -0.0010, -0.0009, -0.0007],
         [-0.0009, -0.0009, -0.0008, -0.0006, -0.0004]],

        [[-0.0010, -0.0011, -0.0009, -0.0009, -0.0011],
         [-0.0009, -0.0009, -0.0009, -0.0010, -0.0011],
         [-0.0010, -0.0011, -0.0013, -0.0012, -0.0012],
         [-0.0011, -0.0013, -0.0013, -0.0012, -0.0011],
         [-0.0012, -0.0012, -0.0012, -0.0010, -0.0008]],

        [[-0.0013, -0.0014, -0.0012, -0.0013, -0.0015],
         [-0.0013, -0.0013, -0.0013, -0.0014, -0.0016],
         [-0.0014, -0.0015, -0.0016, -0.0016, -0.0016],
         [-0.0015, -0.0016, -0.0016, -0.0016, -0.0015],
         [-0.0015, -0.0015, -0.0014, -0.0013, -0.0011]]])
torch.Size([6])
tensor(0.0026)
torch.Size([10, 6, 5, 5])
tensor([[[ 1.3255e-04,  1.3354e-05,  2.8905e-05, -2.1223e-04, -2.1434e-0

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc0 = nn.Linear(3*4*4, 20)
        self.fc01 = nn.Linear(20, 10)
#         self.fc1 = nn.Linear(3 * 16 * 16, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.pool(x)
        x = self.pool(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc0(x))
#         x = F.relu(self.fc2(x))
        x = self.fc01(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
        return x

# class Net(nn.Module):
#                 def __init__(self):
#                         super().__init__()
#                         self.pool = nn.MaxPool2d(2, 2)

#                         self.fc1 = nn.Linear(3 * 8 * 8, 120)
#                         self.fc2 = nn.Linear(120, 84)
#                         self.fc3 = nn.Linear(84, 10)

#                 def forward(self, x):
#                         x = self.pool(x)
#                         x = self.pool(x)
#                         x = torch.flatten(x, 1) # flatten all dimensions except batch
#                         x = self.fc1(x)
#                         x = self.fc2(x)
#                         x = self.fc3(x)
#                         return x

model = Net()

In [52]:
nvar = 10160
x = .1 * np.ones((nvar,1))
x_torch = torch.from_numpy(x).cuda()
torch.nn.utils.vector_to_parameters(x_torch, model.parameters())

lst = list(model.parameters())

for i in range(len(lst)):
#     print(lst[i].grad.shape)
    if i == 0:
        print(lst[0])

Parameter containing:
tensor([[0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        ...,
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000]],
       device='cuda:0', dtype=torch.float64, requires_grad=True)


In [47]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 196 == 195:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 196))
            running_loss = 0.0
            acc = (outputs.max(1)[1] == labels).sum().item()/labels.size(0)
            print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, running_loss, acc ))

print('Finished Training')

[1,   196] loss: 1.884
[1,   196] loss: 0.000 acc: 0.350
[2,   196] loss: 1.874
[2,   196] loss: 0.000 acc: 0.362
[3,   196] loss: 1.865
[3,   196] loss: 0.000 acc: 0.362
[4,   196] loss: 1.855
[4,   196] loss: 0.000 acc: 0.362
[5,   196] loss: 1.847
[5,   196] loss: 0.000 acc: 0.375
[6,   196] loss: 1.838
[6,   196] loss: 0.000 acc: 0.362
[7,   196] loss: 1.831
[7,   196] loss: 0.000 acc: 0.362
[8,   196] loss: 1.823
[8,   196] loss: 0.000 acc: 0.375
[9,   196] loss: 1.816
[9,   196] loss: 0.000 acc: 0.388
[10,   196] loss: 1.809
[10,   196] loss: 0.000 acc: 0.388
[11,   196] loss: 1.802
[11,   196] loss: 0.000 acc: 0.388
[12,   196] loss: 1.796
[12,   196] loss: 0.000 acc: 0.388
[13,   196] loss: 1.790
[13,   196] loss: 0.000 acc: 0.388
[14,   196] loss: 1.784
[14,   196] loss: 0.000 acc: 0.375
[15,   196] loss: 1.779
[15,   196] loss: 0.000 acc: 0.375
[16,   196] loss: 1.773
[16,   196] loss: 0.000 acc: 0.375
[17,   196] loss: 1.768
[17,   196] loss: 0.000 acc: 0.375
[18,   196] los

In [44]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
#         if i >= 1:
#             break

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss = loss.item()
        
        acc = (outputs.max(1)[1] == labels).sum().item()/labels.size(0)
        
        
        print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, running_loss, acc ))

print('Finished Training')

[1,     1] loss: 2.150 acc: 0.215
[1,     2] loss: 2.125 acc: 0.270
[1,     3] loss: 2.120 acc: 0.266
[1,     4] loss: 2.166 acc: 0.219
[1,     5] loss: 2.107 acc: 0.273
[1,     6] loss: 2.129 acc: 0.277
[1,     7] loss: 2.136 acc: 0.273
[1,     8] loss: 2.190 acc: 0.238
[1,     9] loss: 2.154 acc: 0.234
[1,    10] loss: 2.116 acc: 0.281
[1,    11] loss: 2.122 acc: 0.246
[1,    12] loss: 2.132 acc: 0.262
[1,    13] loss: 2.145 acc: 0.227
[1,    14] loss: 2.131 acc: 0.273
[1,    15] loss: 2.127 acc: 0.266
[1,    16] loss: 2.158 acc: 0.254
[1,    17] loss: 2.153 acc: 0.238
[1,    18] loss: 2.161 acc: 0.207
[1,    19] loss: 2.094 acc: 0.316
[1,    20] loss: 2.151 acc: 0.211
[1,    21] loss: 2.138 acc: 0.246
[1,    22] loss: 2.127 acc: 0.289
[1,    23] loss: 2.113 acc: 0.266
[1,    24] loss: 2.124 acc: 0.227
[1,    25] loss: 2.133 acc: 0.262
[1,    26] loss: 2.161 acc: 0.180
[1,    27] loss: 2.172 acc: 0.152
[1,    28] loss: 2.132 acc: 0.254
[1,    29] loss: 2.140 acc: 0.234
[1,    30] los

[2,    49] loss: 2.086 acc: 0.281
[2,    50] loss: 2.075 acc: 0.250
[2,    51] loss: 2.107 acc: 0.250
[2,    52] loss: 2.089 acc: 0.250
[2,    53] loss: 2.076 acc: 0.328
[2,    54] loss: 2.041 acc: 0.281
[2,    55] loss: 2.090 acc: 0.254
[2,    56] loss: 2.103 acc: 0.234
[2,    57] loss: 2.091 acc: 0.258
[2,    58] loss: 2.080 acc: 0.281
[2,    59] loss: 2.089 acc: 0.270
[2,    60] loss: 2.077 acc: 0.266
[2,    61] loss: 2.104 acc: 0.266
[2,    62] loss: 2.074 acc: 0.293
[2,    63] loss: 2.088 acc: 0.219
[2,    64] loss: 2.092 acc: 0.227
[2,    65] loss: 2.082 acc: 0.293
[2,    66] loss: 2.125 acc: 0.238
[2,    67] loss: 2.086 acc: 0.250
[2,    68] loss: 2.105 acc: 0.258
[2,    69] loss: 2.113 acc: 0.254
[2,    70] loss: 2.080 acc: 0.266
[2,    71] loss: 2.068 acc: 0.281
[2,    72] loss: 2.069 acc: 0.266
[2,    73] loss: 2.097 acc: 0.242
[2,    74] loss: 2.056 acc: 0.312
[2,    75] loss: 2.095 acc: 0.285
[2,    76] loss: 2.113 acc: 0.219
[2,    77] loss: 2.041 acc: 0.297
[2,    78] los

[3,    96] loss: 1.998 acc: 0.309
[3,    97] loss: 2.021 acc: 0.301
[3,    98] loss: 2.058 acc: 0.281
[3,    99] loss: 1.992 acc: 0.309
[3,   100] loss: 2.033 acc: 0.285
[3,   101] loss: 2.003 acc: 0.328
[3,   102] loss: 1.981 acc: 0.320
[3,   103] loss: 2.008 acc: 0.281
[3,   104] loss: 2.045 acc: 0.297
[3,   105] loss: 1.995 acc: 0.316
[3,   106] loss: 2.055 acc: 0.262
[3,   107] loss: 2.056 acc: 0.281
[3,   108] loss: 2.045 acc: 0.273
[3,   109] loss: 2.041 acc: 0.254
[3,   110] loss: 2.039 acc: 0.301
[3,   111] loss: 2.048 acc: 0.273
[3,   112] loss: 2.050 acc: 0.211
[3,   113] loss: 2.000 acc: 0.301
[3,   114] loss: 2.054 acc: 0.258
[3,   115] loss: 2.028 acc: 0.270
[3,   116] loss: 2.046 acc: 0.238
[3,   117] loss: 2.001 acc: 0.266
[3,   118] loss: 2.031 acc: 0.285
[3,   119] loss: 2.010 acc: 0.312
[3,   120] loss: 2.065 acc: 0.301
[3,   121] loss: 2.027 acc: 0.285
[3,   122] loss: 2.031 acc: 0.289
[3,   123] loss: 2.035 acc: 0.305
[3,   124] loss: 2.009 acc: 0.285
[3,   125] los

[4,   143] loss: 1.945 acc: 0.336
[4,   144] loss: 1.963 acc: 0.324
[4,   145] loss: 2.023 acc: 0.301
[4,   146] loss: 2.035 acc: 0.273
[4,   147] loss: 2.023 acc: 0.273
[4,   148] loss: 2.012 acc: 0.285
[4,   149] loss: 2.078 acc: 0.250
[4,   150] loss: 1.989 acc: 0.312
[4,   151] loss: 1.973 acc: 0.293
[4,   152] loss: 1.960 acc: 0.348
[4,   153] loss: 1.957 acc: 0.320
[4,   154] loss: 2.022 acc: 0.230
[4,   155] loss: 2.024 acc: 0.273
[4,   156] loss: 2.053 acc: 0.262
[4,   157] loss: 1.999 acc: 0.352
[4,   158] loss: 2.030 acc: 0.293
[4,   159] loss: 1.916 acc: 0.371
[4,   160] loss: 2.067 acc: 0.277
[4,   161] loss: 2.041 acc: 0.297
[4,   162] loss: 1.981 acc: 0.289
[4,   163] loss: 2.029 acc: 0.285
[4,   164] loss: 1.980 acc: 0.293
[4,   165] loss: 2.002 acc: 0.277
[4,   166] loss: 1.945 acc: 0.320
[4,   167] loss: 2.053 acc: 0.285
[4,   168] loss: 2.020 acc: 0.258
[4,   169] loss: 2.012 acc: 0.266
[4,   170] loss: 1.996 acc: 0.266
[4,   171] loss: 1.976 acc: 0.246
[4,   172] los

[5,   193] loss: 1.919 acc: 0.359
[5,   194] loss: 2.005 acc: 0.293
[5,   195] loss: 1.974 acc: 0.293
[5,   196] loss: 2.097 acc: 0.300
[6,     1] loss: 1.956 acc: 0.281
[6,     2] loss: 1.954 acc: 0.332
[6,     3] loss: 1.948 acc: 0.332
[6,     4] loss: 1.981 acc: 0.301
[6,     5] loss: 1.920 acc: 0.332
[6,     6] loss: 1.928 acc: 0.316
[6,     7] loss: 1.948 acc: 0.293
[6,     8] loss: 2.060 acc: 0.262
[6,     9] loss: 1.980 acc: 0.301
[6,    10] loss: 1.946 acc: 0.320
[6,    11] loss: 1.960 acc: 0.312
[6,    12] loss: 1.940 acc: 0.297
[6,    13] loss: 1.986 acc: 0.258
[6,    14] loss: 1.976 acc: 0.312
[6,    15] loss: 1.947 acc: 0.328
[6,    16] loss: 2.019 acc: 0.297
[6,    17] loss: 1.978 acc: 0.277
[6,    18] loss: 2.002 acc: 0.262
[6,    19] loss: 1.923 acc: 0.340
[6,    20] loss: 1.993 acc: 0.289
[6,    21] loss: 1.982 acc: 0.328
[6,    22] loss: 1.950 acc: 0.340
[6,    23] loss: 1.925 acc: 0.266
[6,    24] loss: 1.951 acc: 0.324
[6,    25] loss: 1.947 acc: 0.289
[6,    26] los

[7,    47] loss: 1.996 acc: 0.277
[7,    48] loss: 1.952 acc: 0.301
[7,    49] loss: 1.981 acc: 0.324
[7,    50] loss: 1.948 acc: 0.285
[7,    51] loss: 1.970 acc: 0.312
[7,    52] loss: 1.993 acc: 0.277
[7,    53] loss: 1.960 acc: 0.340
[7,    54] loss: 1.925 acc: 0.309
[7,    55] loss: 1.966 acc: 0.312
[7,    56] loss: 1.974 acc: 0.277
[7,    57] loss: 1.989 acc: 0.273
[7,    58] loss: 1.977 acc: 0.293
[7,    59] loss: 2.013 acc: 0.301
[7,    60] loss: 1.970 acc: 0.309
[7,    61] loss: 2.011 acc: 0.277
[7,    62] loss: 1.938 acc: 0.320
[7,    63] loss: 1.982 acc: 0.227
[7,    64] loss: 1.980 acc: 0.246
[7,    65] loss: 1.993 acc: 0.301
[7,    66] loss: 2.047 acc: 0.281
[7,    67] loss: 1.958 acc: 0.305
[7,    68] loss: 1.988 acc: 0.312
[7,    69] loss: 1.975 acc: 0.320
[7,    70] loss: 1.965 acc: 0.301
[7,    71] loss: 1.941 acc: 0.340
[7,    72] loss: 1.945 acc: 0.344
[7,    73] loss: 1.990 acc: 0.266
[7,    74] loss: 1.930 acc: 0.344
[7,    75] loss: 1.977 acc: 0.316
[7,    76] los

[8,    95] loss: 1.870 acc: 0.359
[8,    96] loss: 1.880 acc: 0.375
[8,    97] loss: 1.929 acc: 0.320
[8,    98] loss: 1.957 acc: 0.309
[8,    99] loss: 1.878 acc: 0.332
[8,   100] loss: 1.927 acc: 0.336
[8,   101] loss: 1.927 acc: 0.355
[8,   102] loss: 1.844 acc: 0.367
[8,   103] loss: 1.881 acc: 0.320
[8,   104] loss: 1.974 acc: 0.281
[8,   105] loss: 1.892 acc: 0.328
[8,   106] loss: 1.977 acc: 0.336
[8,   107] loss: 1.979 acc: 0.324
[8,   108] loss: 1.946 acc: 0.320
[8,   109] loss: 1.957 acc: 0.273
[8,   110] loss: 1.964 acc: 0.324
[8,   111] loss: 1.952 acc: 0.289
[8,   112] loss: 1.960 acc: 0.270
[8,   113] loss: 1.881 acc: 0.352
[8,   114] loss: 1.965 acc: 0.301
[8,   115] loss: 1.928 acc: 0.309
[8,   116] loss: 1.956 acc: 0.277
[8,   117] loss: 1.896 acc: 0.312
[8,   118] loss: 1.932 acc: 0.316
[8,   119] loss: 1.908 acc: 0.328
[8,   120] loss: 1.995 acc: 0.320
[8,   121] loss: 1.923 acc: 0.336
[8,   122] loss: 1.952 acc: 0.320
[8,   123] loss: 1.920 acc: 0.371
[8,   124] los

[9,   145] loss: 1.968 acc: 0.320
[9,   146] loss: 1.962 acc: 0.289
[9,   147] loss: 1.950 acc: 0.340
[9,   148] loss: 1.939 acc: 0.320
[9,   149] loss: 2.033 acc: 0.258
[9,   150] loss: 1.918 acc: 0.352
[9,   151] loss: 1.892 acc: 0.301
[9,   152] loss: 1.870 acc: 0.379
[9,   153] loss: 1.871 acc: 0.340
[9,   154] loss: 1.945 acc: 0.270
[9,   155] loss: 1.965 acc: 0.312
[9,   156] loss: 1.998 acc: 0.293
[9,   157] loss: 1.921 acc: 0.352
[9,   158] loss: 1.968 acc: 0.324
[9,   159] loss: 1.828 acc: 0.387
[9,   160] loss: 2.023 acc: 0.301
[9,   161] loss: 1.970 acc: 0.305
[9,   162] loss: 1.903 acc: 0.340
[9,   163] loss: 1.950 acc: 0.309
[9,   164] loss: 1.902 acc: 0.312
[9,   165] loss: 1.943 acc: 0.320
[9,   166] loss: 1.847 acc: 0.352
[9,   167] loss: 1.998 acc: 0.309
[9,   168] loss: 1.954 acc: 0.297
[9,   169] loss: 1.933 acc: 0.305
[9,   170] loss: 1.923 acc: 0.293
[9,   171] loss: 1.899 acc: 0.297
[9,   172] loss: 1.905 acc: 0.348
[9,   173] loss: 1.924 acc: 0.309
[9,   174] los

[10,   188] loss: 2.005 acc: 0.293
[10,   189] loss: 1.891 acc: 0.320
[10,   190] loss: 1.914 acc: 0.348
[10,   191] loss: 1.911 acc: 0.293
[10,   192] loss: 1.903 acc: 0.297
[10,   193] loss: 1.851 acc: 0.367
[10,   194] loss: 1.949 acc: 0.324
[10,   195] loss: 1.900 acc: 0.320
[10,   196] loss: 2.058 acc: 0.287
Finished Training


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc0 = nn.Linear(3*4*4, 20)
        self.fc01 = nn.Linear(20, 10)
#         self.fc1 = nn.Linear(3 * 16 * 16, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.pool(x)
        x = self.pool(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc0(x))
#         x = F.relu(self.fc2(x))
        x = self.fc01(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
        return x

# class Net(nn.Module):
#                 def __init__(self):
#                         super().__init__()
#                         self.pool = nn.MaxPool2d(2, 2)

#                         self.fc1 = nn.Linear(3 * 8 * 8, 120)
#                         self.fc2 = nn.Linear(120, 84)
#                         self.fc3 = nn.Linear(84, 10)

#                 def forward(self, x):
#                         x = self.pool(x)
#                         x = self.pool(x)
#                         x = torch.flatten(x, 1) # flatten all dimensions except batch
#                         x = self.fc1(x)
#                         x = self.fc2(x)
#                         x = self.fc3(x)
#                         return x

model = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        if i >= 1:
            break
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss = loss.item()
        
        acc = (outputs.max(1)[1] == labels).sum().item()/labels.size(0)
        
        
        print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, running_loss, acc ))

print('Finished Training')

In [ ]:
# zero the parameter gradients
outputs = model(inputs)
loss = criterion(outputs, labels)
loss.backward()
parameter_lst = list(model.parameters())

for i in range(len(parameter_lst)):
    print(parameter_lst[i].shape)

for i in range(len(parameter_lst)):
    if parameter_lst[i].grad != None:
        print(parameter_lst[i].grad.shape)
    else:
        print("None")

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    acc = 0.0
    for i, data in enumerate(trainloader, 0):
        
        if i >= 1:
            break
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        print(inputs.shape)
        print(inputs[0][0])


        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        
#         print(outputs.shape)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss = loss.item()
        
        acc = (outputs.max(1)[1] == labels).sum().item()/labels.size(0)
        
        
        print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, running_loss, acc ))

print('Finished Training')

In [ ]:
lst = list(model.parameters())
for i in range(len(lst)):
#     print(lst[i].shape)
    print(lst[i].grad.shape)
    if i == 0:
        print(lst[0].grad[0])

inputs, labels = data
outputs = model(inputs)
loss = criterion(outputs, labels)
loss.backward()

lst = list(model.parameters())
for i in range(len(lst)):
#     print(lst[i].shape)
    print(lst[i].grad.shape)
    if i == 0:
        print(lst[0].grad[0])
        


In [ ]:
print(lst[0].grad.shape)
vec = torch.reshape(lst[0].grad,(-1,1)).numpy()
print(vec.shape[0])